In [1]:
# 한컴오피스 자동화 입문1 : 상장 자동제작 feat.파이썬
# https://youtu.be/1pC_UXhiJH8

import win32com.client as win32
import pandas as pd

In [2]:
# hwp = win32.Dispatch("HWPFrame.HwpObject")		# 만약 오류가 발생한다면 : pywintypes.com_error: (-2147352562, '매개 변수의 개수가 잘못되었습니다.', None, None)
hwp0 = win32.gencache.EnsureDispatch("HWPFrame.HwpObject")

# 보안모듈 등록 : 오류난다면 레지스트리 키의 철자, 경로 확인할 것
hwp0.RegisterModule('FilePathCheckDLL', "SecurityModule")

# 한글 최신버전인 경우(예를 들어 2020) 반드시 실행해야 한글 프로그램 창이 보인다.
hwp0.XHwpWindows.Item(0).Visible = True

In [3]:
# hwp = win32.Dispatch("HWPFrame.HwpObject")		# 만약 오류가 발생한다면 : pywintypes.com_error: (-2147352562, '매개 변수의 개수가 잘못되었습니다.', None, None)
hwp1 = win32.gencache.EnsureDispatch("HWPFrame.HwpObject")

# 보안모듈 등록 : 오류난다면 레지스트리 키의 철자, 경로 확인할 것
hwp1.RegisterModule('FilePathCheckDLL', "SecurityModule")

# 한글 최신버전인 경우(예를 들어 2020) 반드시 실행해야 한글 프로그램 창이 보인다.
hwp1.XHwpWindows.Item(0).Visible = True

In [4]:
# 사망자: 7월 ~ 8월
month_start = 7
month_end = 8

In [5]:
# 모든 시트 불러오기 : sheet_name=None
# 헤더(= 컬럼 니름 = 1행) 없으면 : header=None
# 인덱스 만들고 싶지 않으면 : index_col=None
excel = pd.read_excel(f"상속 이전 안내문 발송대상자 명단({month_start}월~{month_end}월).xlsx", sheet_name=None)

# excel

In [6]:
# 시트 이름 확인
excel.keys()

dict_keys(['피상속인명단(7월)', '피상속인명단(8월)'])

In [7]:
# 특정 시트만 불러오기 : excel['대상자명단(5월)']
# excel['대상자명단(5월)']

In [8]:
# 전체 시트를 하나로 합치기(인덱스 무시)
excel_total = pd.concat(excel, ignore_index=True)

# excel_total

In [9]:
excel_total.head()

,순번,수신일,사망신고일,자동차등록번호,주민등록번호,성명,소유구분,주민주소,사용본거지 주소,사용본거지 우편번호,전화번호,핸드폰번호,사망사유,시군구등록일시
0,1,2022.07.02,2022.07.01,109오8765,431201-1239413,송주환,대표소유자,"서울특별시 노원구 상계로1길 82-14, 301동 1504호(상계동, 비콘드림힐아파트)","서울특별시 노원구 상계로1길 82-14, 301동 1504호(상계동, 비콘드림힐아파트)",1693.0,NaN,NaN,사망신고말소,2022.07.01 15.54.11
1,2,2022.07.05,2022.07.04,106나2815,510520-1074714,김병준,대표소유자,"서울특별시 노원구 한글비석로 479, 103동 904호(상계동, 보람아파트)","서울특별시 노원구 한글비석로 479, 103동 904호(상계동, 보람아파트)",1678.0,NaN,NaN,11호통보의거사망말소,2022.07.04 14.08.03
2,3,2022.07.05,2022.07.04,140무8054,940122-2035818,이승미,대표소유자,"서울특별시 노원구 동일로236길 51, 301호(상계동, 성원그린빌라)","서울특별시 노원구 동일로236길 51, 301호(상계동, 성원그린빌라)",1628.0,NaN,010-4258-1994,사망신고말소,2022.07.04 11.49.27
3,4,2022.07.05,2022.07.04,61두5686,580802-1036428,이문세,대표소유자,"서울특별시 노원구 동일로179길 36-7, 1층(공릉동)","서울특별시 노원구 동일로179길 36-7, 1층(공릉동)",1856.0,NaN,010-493-5197,사망신고말소,2022.07.04 09.29.17
4,5,2022.07.06,2022.07.05,07누4461,640831-2029717,박정미,대표소유자,"서울특별시 노원구 누원로 18, 405동 1304호(상계동, 수락리버시티4단지아파트)","서울특별시 노원구 누원로 18, 405동 1304호(상계동, 수락리버시티4단지아파트)",1601.0,NaN,NaN,사망신고말소,2022.07.05 16.37.36


In [10]:
# 특정 열만 추출하기
excel_regnum_name = excel_total[['자동차등록번호', '성명']]
# excel_regnum_name

In [11]:
# 열이름 바꾸기
excel_regnum_name.columns=['등록번호', '소유자']
# excel_regnum_name

In [12]:

hwp0.Open("C:/Dev/InheritNotice/상속 이전 등록 안내문.hwp", "HWP", None)
hwp1.Open("C:/Dev/InheritNotice/운행정지 명령 예고문.hwp", "HWP", None)

True

In [13]:
hwp0.GetFieldList()

'등록번호\x02소유자'

In [14]:
hwp1.GetFieldList()

'등록번호\x02소유자'

In [15]:
# 상속이전 안내문과 운행정지 안내문의 누름틀 필드가 같다는 전제
field_list = [i for i in hwp0.GetFieldList().split('\x02')]

In [16]:
field_list

['등록번호', '소유자']

In [17]:
# 총 발송 대상 건수
len(excel_regnum_name)

48

In [18]:
# 1. 상속이전 안내문: 와꾸 복사
hwp0.MovePos(0)        # 맨 처음으로 이동
hwp0.Run('SelectAll')  # 전체 선택
hwp0.Run('Copy')       # hwp.Run('Cut')    hwp.Run('Paste')
hwp0.MovePos(3)        # 맨 끝으로 이동

# (총 건수 - 1) 횟수 "붙여넣기"
for i in range(len(excel_regnum_name)-1):
    hwp0.Run('Paste')
    hwp0.MovePos(3)

hwp0.MovePos(0)

True

In [19]:
# 2. 운행정지 안내문: 와꾸 복사
hwp1.MovePos(0)        # 맨 처음으로 이동
hwp1.Run('SelectAll')  # 전체 선택
hwp1.Run('Copy')       # hwp.Run('Cut')    hwp.Run('Paste')
hwp1.MovePos(3)        # 맨 끝으로 이동

# (총 건수 - 1) 횟수 "붙여넣기"
for i in range(len(excel_regnum_name)-1):
    hwp1.Run('Paste')
    hwp1.MovePos(3)

hwp1.MovePos(0)

True

In [20]:
# 3. 상속이전 / 운행정지 안내문: 누름틀 필드에 값 입력

for page in range(len(excel_regnum_name)):    # 모든 페이지에
    for field in field_list:                  # 각 페이지의 모든 필드(누름틀)
        hwp0.MoveToField(f"{field}{{{{{page}}}}}")  # 커서를 해당 누름틀로 이동
        hwp0.PutFieldText(f"{field}{{{{{page}}}}}", excel_regnum_name[field].iloc[page])
        hwp1.MoveToField(f"{field}{{{{{page}}}}}")  # 커서를 해당 누름틀로 이동
        hwp1.PutFieldText(f"{field}{{{{{page}}}}}", excel_regnum_name[field].iloc[page])

hwp0.MovePos(3)
hwp0.Run("DeleteBack")
hwp0.Run("DeleteBack")
hwp1.MovePos(3)
hwp1.Run("DeleteBack")
hwp1.Run("DeleteBack")

In [21]:
hwp0.SaveAs(f"C:/Dev/InheritNotice/상속 이전 등록 안내문({month_start}월~{month_end}월).hwp")
hwp1.SaveAs(f"C:/Dev/InheritNotice/운행정지 명령 예고문({month_start}월~{month_end}월).hwp")

True

In [22]:
hwp0.Quit()
hwp1.Quit()

### 한글 프로그램 종료 (상속이전 / 운행정지 각각 hwp파일로 저장됨)